In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("TeknolojiYorumları .csv", delimiter=';')
df.head(4)

,Metin,Durum,Alan,Konu
0,Ekran kalitesi çok düşük,0,Teknoloji,ekran
1,elbise iyi,1,Moda,kalite
2,nostaljii...ekrani kucuk ama is goruyor??,1,Teknoloji,ekran
3,mükemmel ama şarjı çabuk bitiyor,2,Teknoloji,şarj


In [2]:
#Sadece metin sütunun kullanılması, konu modelleme için yeterli olacaktır.
yorum_veriseti=df[["Metin"]]
yorum_veriseti.head(4)

,Metin
0,Ekran kalitesi çok düşük
1,elbise iyi
2,nostaljii...ekrani kucuk ama is goruyor??
3,mükemmel ama şarjı çabuk bitiyor


In [3]:
konu_veriseti=df[["Konu"]]
konu_veriseti.head(4)

,Konu
0,ekran
1,kalite
2,ekran
3,şarj


In [4]:
#veri temizleme işlemi için bir fonksiyon hazırlanması

import re 
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


nok_isaretleri= string.punctuation
etkisiz_kelimeler= stopwords.words('turkish')
etkisiz_kelimeler.extend (["bir", "kadar", "sonra"])


def veriTemizleme(metin):
    metin=metin.lower()
    metin=metin.replace("\\n"," ")
    #kesme işareti ve sonrasındaki karakterlerin aldırılması
    metin=re.sub("'(\w+)'", "", metin)
    metin=re.sub("'(\w+)'", "", metin)
    #sayıların kaldırılması
    metin=re.sub("[0-9]+", "", metin)
    #noktalama işaretlerinin kaldırılması
    metin="".join(list(map(lambda x:x if x not in nok_isaretleri else " " , metin)))
    #etkisiz kelimelerin bazılarının kaldırılması
    metin=" ".join([i for i in metin.split() if i not in etkisiz_kelimeler])
    #metinde tek kalan harflerin çıkarılması
    metin=" ".join([i for i in metin.split() if len(i) > 1])

    return metin


<>:19: SyntaxWarning: invalid escape sequence '\w'
<>:20: SyntaxWarning: invalid escape sequence '\w'
<>:19: SyntaxWarning: invalid escape sequence '\w'
<>:20: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Gulseren\AppData\Local\Temp\ipykernel_12504\4275174909.py:19: SyntaxWarning: invalid escape sequence '\w'
  metin=re.sub("'(\w+)'", "", metin)
C:\Users\Gulseren\AppData\Local\Temp\ipykernel_12504\4275174909.py:20: SyntaxWarning: invalid escape sequence '\w'
  metin=re.sub("'(\w+)'", "", metin)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gulseren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Temizlenmemiş veri seti
print("Temizlenmemiş Veri Seti:")
print(df["Metin"].head(10))

# Temizlenmiş veri seti
print("\nTemizlenmiş Veri Seti:")
temizlenen_metin = yorum_veriseti["Metin"].apply(veriTemizleme)
print(temizlenen_metin.head(10))

Temizlenmemiş Veri Seti:
0                             Ekran kalitesi çok düşük
1                                           elbise iyi
2            nostaljii...ekrani kucuk ama is goruyor??
3                     mükemmel ama şarjı çabuk bitiyor
4              sesi rahatsiz ediyor ama çocuklar sevdi
5                    Ürün güzel;bataryası çok gidiyor.
6    Tam bir nostalji yaratıyor. Sesi bile yetti ba...
7    ürün çom güzel eğlenceli içerisinde 500 tane o...
8    o kadar çabuk şarjı bitiyo ki , sırf çocuğumla...
9    Bir heves aldık ama çok gereksiz, oynamak keyi...
Name: Metin, dtype: object

Temizlenmiş Veri Seti:
0                                 ekran kalitesi düşük
1                                           elbise iyi
2                    nostaljii ekrani kucuk is goruyor
3                         mükemmel şarjı çabuk bitiyor
4                  sesi rahatsiz ediyor çocuklar sevdi
5                         ürün güzel bataryası gidiyor
6          tam nostalji yaratıyor sesi bile 

In [6]:
# Tokenize etme
yorum_veriseti["temizlenen_metin_token"] = temizlenen_metin.apply(lambda x: x.split())
yorum_veriseti.head(10)

C:\Users\Gulseren\AppData\Local\Temp\ipykernel_12504\268001877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yorum_veriseti["temizlenen_metin_token"] = temizlenen_metin.apply(lambda x: x.split())


,Metin,temizlenen_metin_token
0,Ekran kalitesi çok düşük,"[ekran, kalitesi, düşük]"
1,elbise iyi,"[elbise, iyi]"
2,nostaljii...ekrani kucuk ama is goruyor??,"[nostaljii, ekrani, kucuk, is, goruyor]"
3,mükemmel ama şarjı çabuk bitiyor,"[mükemmel, şarjı, çabuk, bitiyor]"
4,sesi rahatsiz ediyor ama çocuklar sevdi,"[sesi, rahatsiz, ediyor, çocuklar, sevdi]"
5,Ürün güzel;bataryası çok gidiyor.,"[ürün, güzel, bataryası, gidiyor]"
6,Tam bir nostalji yaratıyor. Sesi bile yetti ba...,"[tam, nostalji, yaratıyor, sesi, bile, yetti, ..."
7,ürün çom güzel eğlenceli içerisinde 500 tane o...,"[ürün, çom, güzel, eğlenceli, içerisinde, tane..."
8,"o kadar çabuk şarjı bitiyo ki , sırf çocuğumla...","[çabuk, şarjı, bitiyo, sırf, çocuğumla, çocukl..."
9,"Bir heves aldık ama çok gereksiz, oynamak keyi...","[heves, aldık, gereksiz, oynamak, keyifli, değ..."


In [17]:
#Bu matris, her bir belgedeki (yorumdaki) kelime frekanslarını içerir.


import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(analyzer='word')
data = cv.fit_transform(temizlenen_metin)
text_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names_out())
text_dtm.index = yorum_veriseti.index
text_dtm.head()


,acman,adana,ade,adet,adi,ailece,aksam,akşam,alacaklar,alacaksanız,...,şarjının,şarzı,şarş,şekilde,şeyi,şikayet,şişme,şişti,şletimi,şuan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
